# Dissertation Code - LSTM Music Generation System

## Import project dependencies

In [ ]:
import re
from tqdm.notebook import trange, tqdm
import os
from music21 import converter, note, chord, instrument, stream, duration, scale, interval, pitch
import numpy as np
from sklearn.utils import shuffle
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.callbacks import ModelCheckpoint, EarlyStopping
from matplotlib import pyplot
import pickle
from keras.models import load_model
import fluidsynth
import subprocess
import random

print("MODULES LOADED")

## Functions to be used

In [ ]:
"""
Initialise some dictionaries
"""
note2inter = {"C": 0,
              "C#": 1,
              "D": 2,
              "E-": 3,
              "E": 4, 
              "F": 5, 
              "F#": 6, 
              "G": 7, 
              "G#": 8, 
              "A": 9,
              "B-": 10,
              "B": 11}

inter2note = {0: "C",
              1: "C#",
              2: "D",
              3: "E-",
              4: "E",
              5: "F",
              6: "F#",
              7: "G",
              8: "G#",
              9: "A",
              10: "B-",
              11: "B"}

dur2string = {0.25: "1/16", 
              0.5: "1/8", 
              1.0: "1/4", 
              2.0: "1/2"}

dur2float = {"1/16" : 0.25,
             "1/8" : 0.5,
             "1/4" : 1.0,
             "1/2" : 2.0}

"""
This function takes in an array/list and a value.
It compares this value to the elements in the array ,
and returns the element closest to the given value (floored).
"""
def find_nearest(array, value):
    
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    
    return array[idx]


"""
This function creates an integer representation of a given scale
The user must input a base note (sharps instead of flats),
and whether they wish the scale to be minor or major
"""
def scale_create():
    
    p = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    key = note2inter[str(input("what key?: ")).upper()]
    m_or_m = str(input("Minor or Major?: "))
    major = [p[key], p[key+2], p[key+4], p[key+5], p[key+7], p[key+9], p[key+11]]
    minor = [p[key], p[key+2], p[key+3], p[key+5], p[key+7], p[key+8], p[key+10]]
    
    if m_or_m.lower() == "major":
        scale = major
    elif m_or_m.lower() == "minor":
        scale = minor
    else:
        print("Please enter a valid key")
        
    return scale


"""
This function takes in an array and a music21 note object.
The array should be an integer representation of a major or minor scale.
The function will then transpose the given note to fit the given scale.
The transposed note is returned.
"""
def scale_notes(scale, note):
    tmp = re.split('(\d+)',note)
    before, octave = tmp[0], tmp[1]
    transpose = note2inter[before]
    final = inter2note[find_nearest(scale, transpose)] + octave
    return final


"""
This function takes in an array and a music21 chord object.
The array should be an integer representation of a major or minor scale.
The function will then transpose the notes in the chord to fit the given scale.
The transposed chord is returned.
"""
def scale_chords(scale, chord):
    transposed = []
    for note in chord.normalOrder:
        transposed.append(find_nearest(scale, note))
        
    if(len(set(transposed)) == 1):
        final = inter2note[(transposed[0])] + "4"
    else:
        final = '.'.join(str(n) for n in transposed)
        
    return final


"""
This function parses all of the MIDI files in a given directory and transcribes their information to an array
All notes and chords are transposed to a specified scale given by user input
All durations are changed to the nearest 1/32, 1/16, 1/8, etc.
"""
def midi_parse(DATADIR):
    
    print("--- PARSING MIDI ---")
    file_num, notes = 0, [] 
    scale = scale_create()
    lengths = np.array([0.25, 0.5, 1, 2])
    files = [os.path.join(DATADIR,fle) for fle in os.listdir(DATADIR) if fle.endswith(".mid")]
    random.shuffle(files)
    
    for file in tqdm(files, "PARSING"):
        
        file_num += 1
        midi_notes, current_parse = [], []
        midi_stream = converter.parseFile(os.path.join(DATADIR, file)) # Convert MIDI file to music21 stream
        piano_parts = []
        instr = instrument.Piano
        
        # Flatten multi-track MIDI
        try:
            for part in instrument.partitionByInstrument(midi_stream):
                if isinstance(part.getInstrument(), instr):
                    current_parse = part[0].recurse()
        except:
            current_parse = midi_stream.flat.notes
    
        # Go through the music21 stream and append the objects to a usable array
        for element in current_parse:
            if isinstance(element, note.Note): 
                check = element.duration.quarterLength
                dur = dur2string[find_nearest(lengths, check)]
                scaled = scale_notes(scale, str(element.pitch))
                midi_notes.append((scaled)+ "_" + str(dur))
                
            if isinstance(element, chord.Chord):                 
                check = element.duration.quarterLength
                dur = dur2string[find_nearest(lengths, check)]
                scaled = scale_chords(scale, element)
                midi_notes.append(scaled+ "_" +str(dur))
                
            elif isinstance(element, note.Rest):                
                check = element.duration.quarterLength
                dur = dur2string[find_nearest(lengths, check)]
                midi_notes.append(("rest")+ "_" +str(dur))

        notes.extend(midi_notes) # Add objects from this file to overall array of notes 
        print("FILE {:3}".format(str(file_num)), end="")
        print(" =   {:100} ".format(file), end="")
        print("NOTES: " + str(len(midi_notes)))
        
    print("\nTOTAL FILES : " + str(file_num))
    print("TOTAL NOTES : " + str(len(notes)))
    
    return notes


"""
This function takes an array of notes and encodes them for use in a nueral network.
"""
def midi_encode(notes, note_info, seq_len):
    print("--- ENCODING MIDI ---")
    note_set, n_vocab = note_info[0], note_info[1]
    seq_in, seq_out = [], []
    X_network, y_network = [], []
    note2int = dict([(note, intt) for intt, note in enumerate(sorted(note_set))]) # Create dict to map note to int
    
    for i in tqdm(range(0, len(notes) - seq_len), desc="ENCODING"):
        seq_in = notes[i:i + seq_len]  # The 100 notes before the output note
        seq_out = notes[i + seq_len]  # The output note
        X_network.append([note2int[unmapped] for unmapped in seq_in])  # Map each input to int and append
        y_network.append(note2int[seq_out])  # Map output to int and append
    
    X_network = np.reshape(X_network, (len(X_network), seq_len, 1))
    X_network = X_network / float(n_vocab) # Normalise input
    
    X_network, y_network = shuffle(X_network, y_network) # Shuffle Data
    y_network = to_categorical(y_network) # Onehot encode
    
    return X_network, y_network


"""
This function creats an LSTM model and saves it to a specified file.
"""
def model_create(notes, note_info, X_network, y_network, model_name):
    print("--- CREATING MODEL ---\n")
    note_set, n_vocab = note_info[0], note_info[1]
    model = Sequential()
    model.add(LSTM(512, input_shape=(X_network.shape[1], X_network.shape[2]), return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.5))
    model.add(LSTM(512))
    model.add(Dense(512))
    model.add(Dropout(0.5))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.save(model_name)
    print("MODEL CREATED AT: Model_Data\\" +model_name+ "\n")
               
    return

"""
This function trains a model using encoded data.
The weights are all saved to the weights filepath.
"""
def model_train(X_network, y_network, model_name, epoch):
    print("--- TRAINING MODEL ---\n")
    filepath = "Model_Data\\Weights\\{epoch:02d}.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
    model = load_model(model_name)
    history = model.fit(X_network, y_network, epochs=epoch, batch_size=64, callbacks=[checkpoint], validation_split=0.2)


"""
The function takes an array of notes and turns it into a music21 stream. 
This music21 stream is then written to a MIDI file.
"""
def stream2midi(y_prediction, identifier, MIDIDIR):
    
    offset = 0
    output_notes = []
    
    for element in y_prediction:   
        pitch, dur = element.split("_") # Split the string into its object and duration
        dur = dur2float[dur] # Use dict to get quarterlength representation of string
        
        # The element is a chord
        if('.' in pitch) or pitch.isdigit():
            notes_in_chord = pitch.split('.') # Split string into induvidual notes in chord
            notes = []
            # Iterate though the chords notes and add them to a chord object
            for current_note in notes_in_chord: 
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.duration = duration.Duration(dur)
            new_chord.offset = offset
            output_notes.append(new_chord) 
            
        # The element is a rest
        elif pitch == "rest":
            new_rest = note.Rest()
            new_rest.duration = duration.Duration(dur)
            new_note.offset = offset  
            output_notes.append(new_rest) 
            
        # The element is a single note
        else:
            new_note = note.Note(pitch)
            new_note.duration = duration.Duration(dur)
            new_note.offset = offset  
            new_note.storedInstrument = instrument.Piano()  
            output_notes.append(new_note) 
        
        offset += dur # Increase offset of next element by duration of current element

    midi_stream = stream.Stream(output_notes) # Create the music21 stream
    midi_stream.write('midi', fp=MIDIDIR+ "\\" +str(identifier)+ ".mid") # Write the stream to MIDI
    print("NOTES HAVE BEEN WRITTEN TO MIDI FILE AT: " +MIDIDIR+ "\\" +str(identifier)+ ".mid\n")


"""
This function predicts a sequence of notes/chords/rests.
To do this it uses a saved version of the model created earlier
as well as specified weights.
"""
def model_predict(notes, note_info, X_network, model_name, weights, MIDIDIR):
    
    print("--- PREDICTING NOTES ---\n")
    number2gen = int(input("How many MIDI files do you want to generate? "))
    note_set, n_vocab = note_info[0], note_info[1]
    int2note = dict([(intt, note) for intt, note in enumerate(sorted(note_set))]) # Create dict for int -> note
    
    # Load the Model.
    model_predict = load_model(model_name)
    model_predict.load_weights(weights)
    
    # Create a specified number of "predictions".
    for i in range(number2gen):     
        start = np.random.randint(0, len(X_network)-1) # Decide where to start generating our music.
        pattern = np.reshape(X_network[start], len(X_network[start])) # Take 100 notes from the 'start' index.
        y_prediction = [] # Initialise output.

        for note_index in tqdm(range(100), desc="FILE: " +str(i+1)+ " (index "+str(start)+")"):
            X_predict = np.reshape(pattern, (1, len(pattern), 1)) # Reshape pattern so it can be used in the model.
            prediction = model_predict.predict(X_predict, verbose=0) # Predict notes using model.
            # Get most likey note (highest int) from the mapping dictionary.
            index = np.argmax(prediction)
            result = int2note[index]
            y_prediction.append(result)
            # Reconfigure network input for next iteration.
            pattern = np.insert(pattern, len(pattern), (index/ float(n_vocab)))
            pattern = pattern[1:len(pattern)]
        
        stream2midi(y_prediction, start, MIDIDIR) # Turn the prediction into a MIDI file
        
"""
This function converts midi files into listenable wav files.
It does this by taking MIDI files at a specified directory,
plays them using a specified soundfont, 
and renders that to an audio file (.wav)
"""
def midi2wav(in_dir, out_dir, soundfont):
    
    print("--- CONVERTING MIDI TO WAV ---\n")
    
    # Loop through directory containing MIDI files
    for in_file in os.listdir(in_dir):
        identifier = in_file.replace(".mid", "") # Extract name of MIDI file
        out_file = str(identifier)+ ".wav"
        midi_dir = os.path.join(in_dir, in_file)
        wav_dir = os.path.join(out_dir, out_file)
        subprocess.call(['fluidsynth', '-F', wav_dir, soundfont, midi_dir]) # Call command to render audio using FluidSynth.
        
    print("MIDI HAS BEEN CONVERT TO WAV AT: " +str(out_dir)+ "\n")
    

print("FUNCTIONS LOADED")

## Subsystem 1 - Model Creation

In [ ]:
"""
This cell handles model creation, as described in section 4.6.2.
"""

def ss1_model_creation(DATASET_DIR):
    
    # 1. Parse data
    notes = midi_parse(DATADIR)
    note_set = set(notes) # Get rid of dupes
    n_vocab = len(note_set) # The number of unique notes
    note_info = (note_set, n_vocab) # Save info in tuple
    
    # 2. Encode data
    X_network, y_network = midi_encode(notes, note_info, 100)
    
    # 3. Create model
    model_name = "Model_Data\\" + str(input("ENTER MODEL NAME: ")) + ".h5"
    model_create(notes, note_info, X_network, y_network, model_name)
    
    # 4. Train model
    epoch = int(input("ENTER NUMBER OF EPOCHS TO TRAIN FOR: "))
    model_train(X_network, y_network, model_name, epoch)
    
    """
    Serialization
    """
    pickle_out = open("Pickled_Data\\notes.pickle", "wb")
    pickle.dump(notes, pickle_out)
    pickle_out.close()
    
    pickle_out = open("Pickled_Data\\X_network.pickle", "wb")
    pickle.dump(X_network, pickle_out)
    pickle_out.close()
    
    pickle_out = open("Pickled_Data\\y_network.pickle", "wb")
    pickle.dump(y_network, pickle_out)
    pickle_out.close()
    
    pickle_out = open("Model_Data\\model_name.pickle", "wb")
    pickle.dump(model_name, pickle_out)
    pickle_out.close()
    
    print("--- RELEVANT DATA HAS BEEN PICKLED ---")
    print("---           FINISHED             ---")
    
    
    
DATADIR = "C:\\Users\\Lukey\\Music_Project_Test\\Dataset"
ss1_model_creation(DATADIR)

<p style="page-break-after:always;"></p>

## Subsytem 2 - Generation

In [ ]:
"""
This cell handles music generation, as described in section 4.6.3.
"""

def ss2_generation(MIDIDIR, WAVDIR, SFDIR):
    
    """
    LOAD UP VALUES
    """
    pickle_in = open("Pickled_Data\\notes.pickle", "rb")
    notes = pickle.load(pickle_in)
    note_set = set(notes) # Get rid of dupes
    n_vocab = len(note_set) # The number of unique notes
    note_info = (note_set, n_vocab) # Save info in tuple
    
    pickle_in = open("Pickled_Data\\X_network.pickle", "rb")
    X_network = pickle.load(pickle_in)
    
    pickle_in = open("Model_Data\\model_name.pickle", "rb")
    model_name = pickle.load(pickle_in)
    
    # Specify weights to use (look at files for specific value)
    weight_epoch = input("Using weights created at epoch: ")
    weights = "Model_Data\\Weights\\" + weight_epoch + ".hdf5" 
    
    # 1. Predict Sequences
    # 2. Create MIDI
    model_predict(notes, note_info, X_network, model_name, weights, MIDIDIR)
    
    # 3. Render Audio
    midi2wav(MIDIDIR, WAVDIR, SFDIR)

    
MIDIDIR = "C:\\Users\\Lukey\\Music_Project_Test\\Generated_MIDI"
WAVDIR = "C:\\Users\\Lukey\\Music_Project_Test\\Generated_Audio"
SFDIR = "C:\\Users\\Lukey\\Music_Project_Test\\soundfont.sf2"

ss2_generation(MIDIDIR, WAVDIR, SFDIR)